### 1. Import Library

In [1]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

# Connected Database
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass  # Não exibir a senha

%load_ext sql   

### 2. Sttings API keys and Start Youtube

In [2]:
api_key = getpass()
api_service_name = "youtube"
api_version = "v3"

# Defined Charnnels
charnnel_ids = ['UC8mDF5mWNGE-Kpfcvnn0bUg',  #  Me poupe!
                'UCnBNoed6NaPlpL8blj1yy8Q',  # Clube do valor
                'UC_mSfchV-fgpPy-vuwML8_A',  # Gustavo Cerbasi
               ]

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

 ········


### 3. Get charnnel Statistic

In [3]:
def charnnel_stats(youtube, charnnel_ids):
    '''This function return the statistics of charnnels'''
    all_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(charnnel_ids)
    )
    response = request.execute()

    for item in response['items']:
        data = {'Charnnel_name': item['snippet']['title'],
                'Subscribesrs': item['statistics']['subscriberCount'],
                'Viwes': item['statistics']['viewCount'],
                'Total_videos': item['statistics']['videoCount'],
                'Charnnel_ID': item['id'],
                'Playlist_ID': item['contentDetails']['relatedPlaylists']['uploads']
                 }
        all_data.append(data)
        
    return pd.DataFrame(all_data)

In [4]:
charnnel_df = charnnel_stats(youtube, charnnel_ids)

charnnel_df

,Charnnel_name,Subscribesrs,Viwes,Total_videos,Charnnel_ID,Playlist_ID
0,Me Poupe!,7570000,547282805,1467,UC8mDF5mWNGE-Kpfcvnn0bUg,UU8mDF5mWNGE-Kpfcvnn0bUg
1,Gustavo Cerbasi,1020000,38634112,857,UC_mSfchV-fgpPy-vuwML8_A,UU_mSfchV-fgpPy-vuwML8_A
2,Clube do Valor,1040000,54831965,1594,UCnBNoed6NaPlpL8blj1yy8Q,UUnBNoed6NaPlpL8blj1yy8Q


### 4. Get videos IDs

In [5]:
def get_videos_ids(youtube, playlist_id):
    '''This function returns all IDs of videos in the playlist.'''
    def fetch_page(playlist_id, page_token=None):
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=page_token
        )
        return request.execute()
    
    all_videos_id = []
    response = fetch_page(playlist_id)
    
    while response:
        for item in response['items']:
            video_id = {
                "Channel_ID": item['snippet']['channelId'],
                "Video_ID": item['contentDetails']['videoId']
            }
            all_videos_id.append(video_id)
        
        next_page_token = response.get('nextPageToken')
        if next_page_token:
            response = fetch_page(playlist_id, next_page_token)
        else:
            break

    return all_videos_id

In [6]:
playlist_ids = list(charnnel_df['Playlist_ID'])

In [7]:
all_videos_ids = []

for playlist_id in playlist_ids:
    videos_id = get_videos_ids(youtube, playlist_id)
    all_videos_ids.extend(videos_id)  

videos_df = pd.DataFrame(all_videos_ids)

In [8]:
videos_df.head()

,Channel_ID,Video_ID
0,UC8mDF5mWNGE-Kpfcvnn0bUg,Hw7C4-Mh288
1,UC8mDF5mWNGE-Kpfcvnn0bUg,jMyYQ4zP0Ic
2,UC8mDF5mWNGE-Kpfcvnn0bUg,XPaHJ_TSqWk
3,UC8mDF5mWNGE-Kpfcvnn0bUg,7IsiwsXPwMQ
4,UC8mDF5mWNGE-Kpfcvnn0bUg,RIwmvirAKkY


### 5. Get videos Statistics

In [9]:
videos_ids = list(videos_df['Video_ID'][:5])

len(videos_ids)

5

In [10]:
def get_video_detail(youtube, video_ids):
    
    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part='snippet,contentDetails,statistics',
            id= ','.join(videos_ids[i:i+50])
        )
        response = request.execute()
    
        for video in response['items']:
            stats_to_keep = {'snippet': ['channelId', 'channelTitle','title', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favoriteCount','commentCount'],
                             'contentDetails': ['duration', 'caption']   
                            }
            video_info = {}
            video_info['video_id'] = video['id']
            
            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None
    
            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [11]:
video_info_df = get_video_detail(youtube, videos_ids)

In [12]:
video_info_df.head()

,video_id,channelId,channelTitle,title,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,caption
0,Hw7C4-Mh288,UC8mDF5mWNGE-Kpfcvnn0bUg,Me Poupe!,INVESTIR DO ZERO: Aulão completo e descomplicado!,2024-05-21T17:16:21Z,120,5929,0,0,P0D,false
1,jMyYQ4zP0Ic,UC8mDF5mWNGE-Kpfcvnn0bUg,Me Poupe!,Hoje é dia de AULÃO com a NATH para você apren...,2024-06-10T16:00:38Z,5761,605,0,12,PT21S,false
2,XPaHJ_TSqWk,UC8mDF5mWNGE-Kpfcvnn0bUg,Me Poupe!,"Fatura do cartão chegou ai, pessoal...? Dia 20...",2024-06-07T22:05:00Z,6470,326,0,0,PT19S,false
3,7IsiwsXPwMQ,UC8mDF5mWNGE-Kpfcvnn0bUg,Me Poupe!,Venha para o AULÃO com a NATH e comece a mudar...,2024-06-07T15:00:12Z,7543,960,0,7,PT54S,false
4,RIwmvirAKkY,UC8mDF5mWNGE-Kpfcvnn0bUg,Me Poupe!,Acredite! Um elástico pode salvar sua vida fin...,2024-06-06T15:00:32Z,20973,1916,0,39,PT53S,false


### 6. Exploratory Data Analisys

### 7. Connect DataBase